In [2]:
# 16/06/2020
# TASI.AI
# Raphael Mourad

# ARIMA + Exogenous variables = ARIMAX
# ARIMAX is an ARIMA model but in which you can include additional features 
# such as option features to better predict the price.

# https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html

###### IMPORT LIBRARIES AND SET UP PARAMETERS

# Import libraries
import os
import pandas as pd
import numpy as np
import sklearn.metrics as mt
import random
import datetime
from pandas_datareader import DataReader
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_model import ARIMA
from pmdarima import auto_arima
from scipy.stats import norm

/home/mourad/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
# Set up directory
dir="/media/mourad/diskSave/MCF_Toulouse/recherche/ConsulProj/RamCiri"
os.chdir(dir)
print(os.getcwd())

/media/mourad/diskSave/MCF_Toulouse/recherche/ConsulProj/RamCiri


In [4]:
# Parameters
symbol="GOOG" # Set up the symbol you want
kdays=30 # Number of days for forecasting

In [5]:
# Create folder for symbol
symbolFolder="results/ARIMA_optionstats/"+symbol
if(os.path.isdir(symbolFolder)==False):
    os.mkdir(symbolFolder, mode=0o777)

In [5]:
###### LOAD AND PREPROCESS DATA

### STOCK PRICE
# Import price
path="data/stockquotes/samples_L3_stockquotes_sample.csv.gz"
data_stockquotes = pd.read_csv(path, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
data_stockquotes["quotedate"] = pd.to_datetime(data_stockquotes["quotedate"])
#print(data_stockquotes)
print(np.unique(data_stockquotes["symbol"]))

# Choose the stock
data_stockquotes_sel=data_stockquotes[data_stockquotes["symbol"]==symbol]
data_stockquotes_sel=data_stockquotes_sel.sort_values('quotedate')
data_stockquotes_sel=data_stockquotes_sel[["symbol","quotedate","close"]]
#data_stockquotes_sel=data_stockquotes_sel.set_index('quotedate')
print(data_stockquotes_sel)

# Check the absence of missing data 
NAcount=data_stockquotes.isnull().sum().sum()
print("Missing data=",NAcount)

['AAPL' 'AMZN' 'AXP' 'BA' 'CAT' 'DIS' 'GOOG' 'GS' 'HD' 'IBM' 'JNJ' 'JPM'
 'KO' 'MCD' 'MRK' 'MSFT' 'NFLX' 'NKE' 'PFE' 'PG' 'XOM']
      symbol  quotedate        close
23481   GOOG 2015-05-01   537.900024
23882   GOOG 2015-05-04   540.780029
23974   GOOG 2015-05-05   530.799988
23598   GOOG 2015-05-06   524.219971
23944   GOOG 2015-05-07   530.700012
...      ...        ...          ...
23695   GOOG 2020-05-22  1410.420000
23267   GOOG 2020-05-26  1417.020000
23804   GOOG 2020-05-27  1417.840000
23742   GOOG 2020-05-28  1416.730000
23412   GOOG 2020-05-29  1428.920000

[1278 rows x 3 columns]
Missing data= 0


In [7]:
### STOCK OPTION STATISTICS
# Import optionstats
pathOptionStats="data/optionstats/samples_L3_optionstats_sample.csv.gz"
data_optionStats = pd.read_csv(pathOptionStats, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
data_optionStats["quotedate"] = pd.to_datetime(data_optionStats["quotedate"])
print(data_optionStats.columns.values)

# Choose the stock
data_optionStats_sel=data_optionStats[data_optionStats["symbol"]==symbol]
data_optionStats_sel=data_optionStats_sel.sort_values(["quotedate"])
#data_optionStats_sel=data_optionStats_sel.set_index('quotedate')
data_optionStats_sel=pd.DataFrame.drop_duplicates(data_optionStats_sel)
data_optionStats_sel=data_optionStats_sel.drop(columns=["symbol"])
print(data_optionStats_sel[["iv30call","iv30put"]])
#print(data_optionStats_sel)

['symbol' 'quotedate' 'iv30call' 'iv30put' 'iv30mean' 'iv60call' 'iv60put'
 'iv60mean' 'iv90call' 'iv90put' 'iv90mean' 'iv120call' 'iv120put'
 'iv120mean' 'iv150call' 'iv150put' 'iv150mean' 'iv180call' 'iv180put'
 'iv180mean' 'iv360call' 'iv360put' 'iv360mean' 'callvol' 'putvol'
 'totalvol' 'calloi' 'putoi' 'totaloi']
       iv30call  iv30put
202      0.2106   0.2012
34324    0.2089   0.1835
11227    0.2205   0.1993
25867    0.2121   0.2186
17833    0.2114   0.1986
...         ...      ...
21701    0.2622   0.2626
46776    0.2629   0.2587
29196    0.2646   0.2688
7300     0.2701   0.2722
10736    0.2629   0.2579

[1114 rows x 2 columns]


In [7]:
### MERGE PRICE WITH OPTION STATS
priceOptionStats=data_stockquotes_sel.merge(data_optionStats_sel, left_on='quotedate', right_on='quotedate')
priceOptionStats=priceOptionStats.drop(columns=["symbol"])
priceOptionStats=priceOptionStats.set_index('quotedate')
#print(priceOptionStats[["close","iv30call","iv30put"]])
#print(priceOptionStats)

In [ ]:
###### BUILD AND AUTOSELECT ARIMA MODEL

# Split train/test data
priceOptionStatsTrain = priceOptionStats[0:(len(priceOptionStats)-kdays)]
priceOptionStatsTest = priceOptionStats[(len(priceOptionStats)-kdays):len(priceOptionStats)]

priceCloseTrain=priceOptionStatsTrain["close"]
priceCloseTest=priceOptionStatsTest["close"]

exogenousTrain=priceOptionStatsTrain.drop(columns="close")
exogenousTest=priceOptionStatsTest.drop(columns="close")
#print(priceOptionStatsTrain)
#print(priceOptionStatsTest[0:5])

In [ ]:
# Build ARIMA model and finding automatically the best ARIMA model (function called auto_arima)
autoARIMA = auto_arima(priceCloseTrain, seasonal=False, trace=True,
                error_action='ignore', suppress_warnings=True, stepwise=True)
ARIMAbestparams=autoARIMA.order
print(ARIMAbestparams)

In [ ]:
# Ranking of exogenous features one by one using AIC
# The higher the loss the better the feature
print("No exo: \tAIC="+str(round(autoARIMA.aic())))

exogenousVars=exogenousTrain.columns.values
diffAICVars=list()
for k in range(0, len(exogenousVars)):
    vark=exogenousVars[k]
    ARIMAi=auto_arima(priceCloseTrain, exogenous=pd.DataFrame(exogenousTrain[vark]), 
            seasonal=False, trace=False, error_action='ignore', suppress_warnings=True, stepwise=True,
            start_p=ARIMAbestparams[0], d=ARIMAbestparams[1], start_q=ARIMAbestparams[2], 
            max_p=ARIMAbestparams[0], max_d=ARIMAbestparams[1], max_q=ARIMAbestparams[2])
    diffAICVars.append(round(autoARIMA.aic()-ARIMAi.aic()))
    print(vark+": \tAIC="+str(round(ARIMAi.aic()))+"\tdiffAIC=",str(round(autoARIMA.aic()-ARIMAi.aic())))

In [ ]:
###### Assess combination of exogenous variables to find the best combination

# List of best exogenous variable candidates
exoVarSelected=["iv30mean","iv60mean","iv90mean","iv120mean","iv150mean","iv180mean","callvol","putvol"]

# ARIMA with exogenous variables
ARIMAexolist=list()
for k in range(0, len(exoVarSelected)):
    ARIMAexoi=auto_arima(priceCloseTrain, exogenous=pd.DataFrame(exogenousTrain[exoVarSelected[0:(k+1)]]), seasonal=False, trace=False,
            error_action='ignore', suppress_warnings=True, stepwise=True,
            start_p=ARIMAbestparams[0], d=ARIMAbestparams[1], start_q=ARIMAbestparams[2], 
            max_p=ARIMAbestparams[0], max_d=ARIMAbestparams[1], max_q=ARIMAbestparams[2])
    ARIMAexolist.append(ARIMAexoi)
    print("Exo "+str(exoVarSelected[0:(k+1)])+": \tAIC="+str(round(ARIMAexoi.aic()))+"\tdiffAIC="+str(round(autoARIMA.aic()-ARIMAexoi.aic())))

In [ ]:
###### FORECAST AVERAGE PRICE USING ARIMA MODEL

# Forecast with no exogenous variable (for comparison)
future_forecast_noexo = autoARIMA.predict(n_periods=kdays)

# Forecast for all exogenous variables with ARIMA 
# A ARIMA model is built to predict each exogenous variable.
future_forecast_forexo=pd.DataFrame()
for var in exoVarSelected:
    autoARIMAforexoi = auto_arima(exogenousTrain[var], seasonal=False, trace=False,
                error_action='ignore', suppress_warnings=True, stepwise=True)
    future_forecast_forexoi=pd.DataFrame(autoARIMAforexoi.predict(n_periods=kdays))
    future_forecast_forexo=pd.concat([future_forecast_forexo, future_forecast_forexoi], axis=1)
future_forecast_forexo.columns=exoVarSelected

In [ ]:
# Forecast using ARIMA with exogenous variables
RMSEnoexo=np.sqrt(mt.mean_squared_error(priceCloseTest, future_forecast_noexo))
MAEnoexo=mt.mean_absolute_error(priceCloseTest, future_forecast_noexo)

RMSElist=[None] * len(exoVarSelected)
MAElist=[None] * len(exoVarSelected)
ExoVarList=[None] * len(exoVarSelected)
future_forecast_exo_List=[None] * len(exoVarSelected)
conf_int_exo_List=[None] * len(exoVarSelected)
for k in range(0, len(exoVarSelected)):
    future_forecast_exoi, conf_int_exoi = ARIMAexolist[k].predict(n_periods=kdays, return_conf_int=True,alpha=0.05,
                                exogenous=pd.DataFrame(future_forecast_forexo[exoVarSelected[0:(k+1)]]))
    RMSEi=np.sqrt(mt.mean_squared_error(priceCloseTest, future_forecast_exoi))
    RMSElist[k]=RMSEi
    MAEi=mt.mean_absolute_error(priceCloseTest, future_forecast_exoi)
    MAElist[k]=MAEi
    ExoVarList[k]=''.join(exoVarSelected[0:(k+1)])
    future_forecast_exo_List[k]=future_forecast_exoi
    conf_int_exo_List[k]=conf_int_exoi
    #print(str(exoVarSelected[0:(k+1)]))

In [ ]:
# Best models
print("RMSE_noexo: "+str(round(RMSEnoexo,3))+"\tMAE_noexo: "+str(round(MAEnoexo,3))+"\n")

matMetrics=pd.DataFrame()
matMetrics["ExoVar"]=ExoVarList
matMetrics["RMSEexo"]=RMSElist
matMetrics["MAEexo"]=MAElist
print(matMetrics)
print("\nBest model: "+ExoVarList[np.argmin(RMSElist)])

In [ ]:
# Forecast with confidence interval (alpha=5%)
future_forecast_exobest=future_forecast_exo_List[np.argmin(RMSElist)]
conf_int_exobest=conf_int_exo_List[np.argmin(RMSElist)]

future_forecast = pd.DataFrame(future_forecast_exobest,columns=['future_forecast'],index=priceCloseTest.index.values)
priceCloseTest = pd.DataFrame(priceCloseTest)
conf_int=conf_int_exobest

In [ ]:
# Better forecast plots
plotForeCastBetter=symbolFolder+"/plot_ARIMA_forecast_better_"+symbol+".pdf"
fig, axes = pyplot.subplots(2, 1, figsize=(12, 12))

# --------------------- Actual vs. Predicted --------------------------
axes[0].plot(priceCloseTrain, color='blue', label='Training Data')
axes[0].plot(priceCloseTest.index, future_forecast, color='green', linewidth=4, label='Predicted Price')

axes[0].plot(priceCloseTest.index, priceCloseTest, color='red', label='Actual Price')
axes[0].set_title('Prices Prediction')
axes[0].set_xlabel('Dates')
axes[0].set_ylabel('Prices')
axes[0].legend()


# ------------------ Predicted with confidence intervals ----------------
axes[1].plot(priceCloseTrain, color='blue', label='Training Data')
axes[1].plot(priceCloseTest.index, future_forecast, color='green',
             label='Predicted Price')

axes[1].set_title('Prices Predictions & Confidence Intervals')
axes[1].set_xlabel('Dates')
axes[1].set_ylabel('Prices')

conf_int = np.asarray(conf_int)
axes[1].fill_between(priceCloseTest.index, conf_int[:, 0], conf_int[:, 1],
                     alpha=0.9, color='orange', label="95% Confidence Intervals")
axes[1].legend()
pyplot.savefig(plotForeCastBetter, bbox_inches='tight')


In [ ]:
###### PROBABILITY GRAPH FROM ARIMA MODEL

# Note to Rim and Quinn: ARIMA model computes a predicted mean mu at kdays and a 95% confidence interval
# To compute the probability graph (AKA density function in statistics),
# we calculate the Standard Error (SE) from the lower bound of the confidence interval.
# Once we have the mean mu and the SE, then we can plot the probability graph.

# Current price
priceCloseCurrent=priceCloseTrain.iloc[len(priceCloseTrain)-1]

# Extract predicted mean and confidence interval
mu=future_forecast.iloc[kdays-1,0]
lower=conf_int[kdays-1,0]
upper=conf_int[kdays-1,1]
print("mu="+str(round(mu,3)))

# Compute Standard Error (SE) from predicted confidence interval (reverse engineering here ;) )
se=(mu-lower)/norm.ppf(0.975)
print("SE="+str(round(se,3)))

# Probability graph
plotProbabilityGraph=symbolFolder+"/plot_ARIMA_probabilityGraph_"+symbol+"_"+str(kdays)+"days.pdf"
x = np.linspace(mu - 3*se, mu + 3*se, 100)
pyplot.plot(x, norm.pdf(x, mu, se))
pyplot.axvline(x=priceCloseCurrent,color='red',label='Current Price')
pyplot.axvline(x=lower,color='green',label='95% Pred Interval')
pyplot.axvline(x=upper,color='green')
pyplot.xlabel('Stock price ($)', fontsize=18)
pyplot.ylabel('Probability', fontsize=18)
pyplot.legend(loc='upper right')
pyplot.savefig(plotProbabilityGraph, bbox_inches='tight')
pyplot.show()